In [4]:
import pandas as pd
import datetime
import pycountry

# Países con nombres no reconocidos por pycountry
Iso3Map = {
    'P.R. China': 'CHN',
    'R. Korea': 'KOR',
    'Chinese Taipei': 'TWN'
}


def get_iso3(country_name):
    try:
        # Buscar el país en pycountry
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return Iso3Map.get(country_name, 'None')

excel_file = pd.ExcelFile('Bases\Granted_patents_2014-2023_per_technology_field_per_country_of_residence_en.xlsx')

df_panel = pd.DataFrame()

# Iterar sobre cada hoja (cada año)
for sheet_name in excel_file.sheet_names:
    df_patents = df_patents = pd.read_excel('Bases\Granted_patents_2014-2023_per_technology_field_per_country_of_residence_en.xlsx', sheet_name=sheet_name)[3:38]
    column_names = pd.read_excel('Bases\Granted_patents_2014-2023_per_technology_field_per_country_of_residence_en.xlsx', sheet_name=sheet_name).iloc[2]
    df_patents.columns = column_names
    df_patents.drop(df_patents.columns[1], axis=1, inplace=True)
    
    df_patents.rename(columns={df_patents.columns[0]: 'Field of Technology'}, inplace=True)
    df_patents['Field of Technology'] = df_patents['Field of Technology'].fillna(method='ffill')
      
    df_grouped = df_patents.groupby('Field of Technology').sum()
    df_grouped.reset_index(inplace=True)
    
    total_row = df_grouped.iloc[:, 1:].sum()
    total_row['Field of Technology'] = 'Total'
    df_grouped = pd.concat([df_grouped, pd.DataFrame([total_row])], ignore_index=True)
   
    df_grouped['Year'] = sheet_name
    
    # Reorganizar el DataFrame en formato largo (país, tecnología, año, cantidad)
    df_long = df_grouped.melt(id_vars=['Field of Technology', 'Year'], var_name='Country', value_name='Patents')
    
    # Concatenar los datos al panel general
    df_panel = pd.concat([df_panel, df_long], ignore_index=True)

df_panel['Code'] = df_panel['Country'].apply(get_iso3)
df_panel = df_panel[['Country','Code','Year','Field of Technology','Patents']]

df_dummies = pd.get_dummies(df_panel['Field of Technology'], prefix = None)
df_panel = pd.concat([df_panel, df_dummies], axis=1)
df_panel['Year'] = df_panel['Year'].astype(int)

# Variables Exógenas

ArtCient = pd.read_excel('Bases/articuloscient.xlsx')
CH = pd.read_excel('Bases\capitalhumano.xlsx')
EUComer = pd.read_excel('Bases\EUComercio.xlsx')
Inst = pd.read_excel('Bases\Inst.xlsx')
Exog = ArtCient.merge(CH, on = ['Entity','Year','Code'])
Exog = Exog.merge(Inst, on = ['Year','Code'])
Exog = Exog.merge(EUComer, on = ['Code','Year'])
Exog.drop(columns = 'Entity', inplace = True)


# Instrumento
Instru = pd.read_excel('Bases\LegalOrigin.xlsx')

#Unir Variable Endog y Exog
DATA = df_panel.merge(Exog, on = ['Code','Year'])
DATA = DATA.merge(Instru, on = ['Code'])

DATA.to_excel('DATA.xlsx',index=False)

In [5]:
DATA

,Country,Code,Year,Field of Technology,Patents,Chemistry,Electrical engineering,Instruments,Mechanical engineering,Other fields,Total,ArtCientificos,CapitalHumano,Control de la Corrupción,Calidad Regulatoria,Estado de Derecho,Riesgo de Expropiación,Porcent,LegalOrg
0,Austria,AUT,2014,Chemistry,205,1,0,0,0,0,0,12408.16,5010.507324,7.933004,7.964340,8.876872,8.475253,1.662861,German
1,Austria,AUT,2014,Electrical engineering,135,0,1,0,0,0,0,12408.16,5010.507324,7.933004,7.964340,8.876872,8.475253,1.662861,German
2,Austria,AUT,2014,Instruments,111,0,0,1,0,0,0,12408.16,5010.507324,7.933004,7.964340,8.876872,8.475253,1.662861,German
3,Austria,AUT,2014,Mechanical engineering,308,0,0,0,1,0,0,12408.16,5010.507324,7.933004,7.964340,8.876872,8.475253,1.662861,German
4,Austria,AUT,2014,Other fields,132,0,0,0,0,1,0,12408.16,5010.507324,7.933004,7.964340,8.876872,8.475253,1.662861,German
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1573,Iceland,ISL,2018,Electrical engineering,1,0,1,0,0,0,0,706.71,5879.265625,8.611206,7.816491,8.358503,8.723581,0.047556,German
1574,Iceland,ISL,2018,Instruments,32,0,0,1,0,0,0,706.71,5879.265625,8.611206,7.816491,8.358503,8.723581,0.047556,German
1575,Iceland,ISL,2018,Mechanical engineering,10,0,0,0,1,0,0,706.71,5879.265625,8.611206,7.816491,8.358503,8.723581,0.047556,German
1576,Iceland,ISL,2018,Other fields,4,0,0,0,0,1,0,706.71,5879.265625,8.611206,7.816491,8.358503,8.723581,0.047556,German
